In [2]:
import os
import ast
import json
import pandas as pd

from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from datetime import datetime

In [3]:
#############################################
### TRABAJAMOS SOBRE EL ARCHIVO DE JUEGOS ###
#############################################

datos = []            # Definimos/limpiamos una lista que servira como nexo entre los datos y el dataframe.
with open('output_steam_games.json') as archivo:            # Abrimos el archivo json.
    for linea in archivo:
        datos.append(json.loads(linea))         # Agregamos cada linea del json a la lista ya creada.

dfGames = pd.DataFrame(datos)          # Creamos un dataframe con los datos de la lista.
dfGames = dfGames.dropna(axis=0, how='all')           # Eliminamos del dataframe las filas en las que todos sus campos son nulos.

dfGames = dfGames.drop(['publisher', 'title', 'url', 'specs', 'price', 'reviews_url', 'price', 'early_access', 'developer'], axis=1)          # Eliminamos las columnos que no seran de ayuda en el analisis.

dfGames = dfGames.dropna(subset=['release_date'])        # Eliminanos los datos nan de la columna de fechas.

for fila in dfGames['release_date']:
    try:
        dfGames.loc[dfGames['release_date'] == fila, 'nueva_fecha'] = datetime.strptime(fila, "%Y-%m-%d").year         # Guardamos en una nueva columna, las fechas con formato valido.
    except ValueError:
       dfGames.loc[dfGames['release_date'] == fila, 'nueva_fecha'] = None         # Si la fecha no es valida, guardamos un nan en su lugar.
       

dfGames = dfGames.drop(['release_date'], axis = 1)            # Eliminamos la columna que ya no sera de ayuda.
dfGames = dfGames.rename(columns = {'nueva_fecha': 'release_date'})         # Renombramos la nueva columna con el nombre que ya nos hemos familiarizado.
dfGames = dfGames.dropna(subset = ['release_date'])        #Eliminanos los datos nan de la columna de fechas.
dfGames = dfGames.explode('genres')

In [4]:
dfGames.head()

,genres,app_name,tags,id,release_date
88310,Action,Lost Summoner Kitty,"[Strategy, Action, Indie, Casual, Simulation]",761140,2018.0
88310,Casual,Lost Summoner Kitty,"[Strategy, Action, Indie, Casual, Simulation]",761140,2018.0
88310,Indie,Lost Summoner Kitty,"[Strategy, Action, Indie, Casual, Simulation]",761140,2018.0
88310,Simulation,Lost Summoner Kitty,"[Strategy, Action, Indie, Casual, Simulation]",761140,2018.0
88310,Strategy,Lost Summoner Kitty,"[Strategy, Action, Indie, Casual, Simulation]",761140,2018.0


In [5]:
##############################################
### TRABAJAMOS SOBRE EL ARCHIVO DE REVIEWS ###
##############################################

datos=[]            # Definimos/limpiamos una lista que servira como nexo entre los datos y el dataframe.

with open('australian_user_reviews.json') as archivo:           # Abrimos el archivo json.
    for lineas in archivo:
        linea = ast.literal_eval(lineas)            # Ayuda a manejar el "json" sin que el formato traiga problemas.
        datos.append(linea)         # Agregamos cada linea del "json" a la lista ya creada.

dfReviews = pd.DataFrame(datos)         # Creamos un dataframe con los datos de la lista.
dfReviews = dfReviews.drop(['user_url'], axis = 1)            # Eliminamos las columnos que no seran de ayuda en el analisis.


dfReviews = dfReviews.drop_duplicates('user_id')            # Eliminamos los usuarios que esten duplicados.
dfReviews = dfReviews.explode('reviews')          # Se desanidan los reviews, trasladando cada una a una fila diferente.

dfReviews.dropna()          # Eliminamos los registros nan.
dfReviews = dfReviews.dropna(subset = ['reviews'])            # Eliminamos las filas que no tengan reviews.

for key in dfReviews['reviews'].iloc[0].keys():         
    dfReviews[key] = dfReviews['reviews'].apply(lambda x: x[key])         # Toma cada "llave/valor" correspondiente a las reviews y arma columnas aparte para tener los datos ordenados y accesibles.
    
dfReviews = dfReviews.drop(['reviews', 'funny', 'last_edited', 'helpful'], axis = 1)         # Eliminamos la columna REVIEWS, la cual ya fue desanidada y otras que no utilizaremos.

In [6]:
dfReviews.head()

,user_id,posted,item_id,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
0,76561197970982479,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.
0,76561197970982479,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
1,js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...
1,js41637,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...


In [7]:
############################################
### TRABAJAMOS SOBRE EL ARCHIVO DE ITEMS ###
############################################

datos=[]            # Definimos/limpiamos una lista que servira como nexo entre los datos y el dataframe.

with open('australian_users_items.json') as archivo:            # Abrimos el archivo json.
    for lineas in archivo:
        linea = ast.literal_eval(lineas)         # Ayuda a manejar el "json" sin que el formato traiga problemas.
        datos.append(linea)         # Agregamos cada linea del "json" a la lista ya creada.

dfItems = pd.DataFrame(datos)         # Creamos un dataframe con los datos de la lista.
dfItems = dfItems.drop(['user_url', 'steam_id'], axis=1)            # Eliminamos las columnos que no seran de ayuda en el analisis.


dfItems = dfItems.drop_duplicates('user_id')            # Eliminamos los usuarios que esten duplicados.
dfItems = dfItems.drop(dfItems[(dfItems['items_count'] == 0)].index)            # Eliminamos los usuarios que no tengan juegos.
dfItems = dfItems.explode('items')          # Se desanidan los juegos, trasladando cada uno a una fila diferente.

for key in dfItems['items'].iloc[0].keys():         
    dfItems[key] = dfItems['items'].apply(lambda x: x[key])         # Toma cada "llave/valor" correspondiente a los juegos y arma columnas aparte para tener los datos ordenados y accesibles.
    
dfItems = dfItems.drop(['items', 'items_count', 'playtime_2weeks'], axis = 1)         # Eliminamos la columna ITEMS, la cual ya fue desanidada y otras que no utilizaremos.

In [8]:
dfItems.head()

,user_id,item_id,item_name,playtime_forever
0,76561197970982479,10,Counter-Strike,6
0,76561197970982479,20,Team Fortress Classic,0
0,76561197970982479,30,Day of Defeat,7
0,76561197970982479,40,Deathmatch Classic,0
0,76561197970982479,50,Half-Life: Opposing Force,0


In [9]:
#######################
### PRIMER CONSULTA ###
#######################

dfAux = dfItems.groupby(by = 'item_id').playtime_forever.sum()            # Agrupamos los juegos, obteniendo las horas totales de juego.
dfResultadoU = pd.merge(dfGames, dfAux, how = 'inner', left_on='id', right_on = 'item_id')         # Agregamos al dataframe JUEGOS, el total de horas jugadas en c/u.
dfResultadoU = dfResultadoU.dropna(subset=['genres'])        #Eliminanos los datos nan de la columna de fechas.

dfResultadoU = dfResultadoU.groupby(['genres' , 'release_date'])['playtime_forever'].sum().reset_index()           # Agrupamos los datos por genero y año, sumando las horas.
dfResultadoU['max'] = dfResultadoU.groupby('genres')['playtime_forever'].transform(max)           # Detectamos la mayor cantidad de horas y las agregamos a una columna auxiliar.
dfResultadoU = dfResultadoU[dfResultadoU['playtime_forever'] == dfResultadoU['max']]         # Filtramos el dataframe para que traiga solo los generos y el año con la mayor cantidad de horas.
dfResultadoU['release_date'] = dfResultadoU['release_date'].astype('int')         # Converimos el año a formato numerico.

### EXPORTAMOS A .CSV EL DATAFRAME ###
os.makedirs('Resultados', exist_ok = True)
dfResultadoU.to_csv('Resultados/PConsulta.csv')

In [10]:
dfResultadoU.head()

,genres,release_date,playtime_forever,max
26,Action,2012,1085500221,1085500221
59,Adventure,2011,221631891,221631891
71,Animation &amp; Modeling,2015,1345545,1345545
75,Audio Production,2014,455463,455463
104,Casual,2015,81706883,81706883


In [11]:
########################
### SEGUNDA CONSULTA ###
########################

dfResultadoD = pd.merge(dfGames, dfReviews, how = 'inner', left_on='id', right_on='item_id')            # Creamos un nuevo dataframe, agrupando los juegos y las reviews.
dfResultadoD = pd.merge(dfResultadoD, dfItems, how='inner', left_on=['user_id', 'item_id'], right_on=['user_id', 'item_id'])            # Agregamos al mismo dataframe, el tiempo juegado.
dfResultadoD = dfResultadoD.drop(['tags', 'release_date',  'item_id', 'recommend', 'review', 'item_name'], axis = 1)          # Eliminamos las columnas que no seras utilizadas.

for fila in dfResultadoD['posted']:
    try:
        dfResultadoD.loc[dfResultadoD['posted'] == fila, 'nueva_fecha'] = datetime.strptime(fila, 'Posted %B %d, %Y.').year         # Guardamos en una nueva columna, las fechas con formato valido (poseen anio).
    except ValueError:
       dfResultadoD.loc[dfResultadoD['posted'] == fila, 'nueva_fecha'] = None         # Si la fecha no es valida (no posee anio), guardamos un nan en su lugar.

dfResultadoD = dfResultadoD.dropna(subset=['nueva_fecha'])        #Eliminanos los datos nan de la columna de fechas.
dfResultadoD = dfResultadoD.groupby(['user_id', 'nueva_fecha', 'genres'])['playtime_forever'].sum().reset_index()           # Agrupamos los datos por usuario, anio y genero, sumando las horas.
dfResultadoD['max'] = dfResultadoD.groupby(['user_id', 'genres'])['playtime_forever'].transform(sum)           # Sumamos las horas totales por genero de cada jugador y las agregamos a una columna auxiliar.
dfResultadoD = dfResultadoD[dfResultadoD.groupby("genres")["max"].transform(max) == dfResultadoD["max"]]            # Filtramos los datos por la maxima cantidad de horas, dejando solo los que necesitamos para la consulta.
dfResultadoD['nueva_fecha'] = dfResultadoD['nueva_fecha'].astype('int')         # Converimos el año a formato numero.


### EXPORTAMOS A .CSV EL DATAFRAME ###
os.makedirs('Resultados', exist_ok = True)
dfResultadoD.to_csv('Resultados/SConsulta.csv')

In [12]:
dfResultadoD.head()

,user_id,nueva_fecha,genres,playtime_forever,max
10951,76561198059330972,2015,Animation &amp; Modeling,65427,65427
10952,76561198059330972,2015,Education,65427,65427
10955,76561198059330972,2015,Utilities,65427,65427
34805,Kipikinson,2015,Video Production,52768,52768
39998,SambaWarKiddo,2014,Audio Production,7025,7025


In [29]:
##########################
### 3ra y 4ta CONSULTA ###
##########################

dfResultadoT = dfReviews

for fila in dfResultadoT['posted']:
    try:
        dfResultadoT.loc[dfResultadoT['posted'] == fila, 'nueva_fecha'] = datetime.strptime(fila, 'Posted %B %d, %Y.').year         # Guardamos en una nueva columna, las fechas con formato valido (poseen anio).
    except ValueError:
       dfResultadoT.loc[dfResultadoT['posted'] == fila, 'nueva_fecha'] = None         # Si la fecha no es valida (no posee anio), guardamos un nan en su lugar.
       
dfResultadoT = dfResultadoT.dropna(subset=['nueva_fecha'])        #Eliminanos los datos nan de la columna de fechas.


for fila in dfResultadoT['review']:
    text_blob = TextBlob(fila)

    polarity = text_blob.polarity           # Calculamos la polaridad de las reviews.
    subjectivity = text_blob.subjectivity           # Calculamos la subjetividad de las rewiews.
        
    if subjectivity > 0.4:          # Si la subjetividad es mayor a 0.4, lo tomamos como una review valida [...] 
        if polarity > 0:
            dfResultadoT.loc[dfResultadoT['review'] == fila, 'positi'] = 1          # Si la polaridad es mayor a cero, es positiva.
        elif polarity < 0:
            dfResultadoT.loc[dfResultadoT['review'] == fila, 'negati'] = 1          # Si la polaridad es menor a cero, es negativa.
        else:
            dfResultadoT.loc[dfResultadoT['review'] == fila, 'neutro'] = 1          # Si la polaridad es cero, es neutra.
    else:
        dfResultadoT.loc[dfResultadoT['review'] == fila, 'neutro'] = 1          # [...] De lo contrario lo marcamos como neutro.
        
dfResultadoT = dfResultadoT.groupby(["recommend", "nueva_fecha", "item_id"]).sum().reset_index()            # Agrupamos los datos por recomendacion, fecha e item. Sumamos los resultados del analisis.
dfResultadoT = dfResultadoT.drop(['user_id', 'posted', 'review'], axis = 1)         # Eliminamos la columnas que no utilizaremos.
dfResultadoT['nueva_fecha'] = dfResultadoT['nueva_fecha'].astype('int')         # Convertimos los años a integer.
dfResultadoT['neutro'] = dfResultadoT['neutro'].astype('int')         # Convertimos a integer.
dfResultadoT['positi'] = dfResultadoT['positi'].astype('int')         # Convertimos a integer.
dfResultadoT['negati'] = dfResultadoT['negati'].astype('int')         # Convertimos a integer.

dfAux = dfItems.drop(['user_id', 'playtime_forever'], axis = 1)         # Creamos un dataframe auxiliar con los nombres de los juegos.
dfAux = dfAux.drop_duplicates()         # Eliminamos datos duplicados.

dfResultadoT = pd.merge(dfResultadoT, dfAux, how='left', left_on=['item_id'], right_on=['item_id'])            # Agregamos los nombres de los juegos a nuestro dataframe de resultado.

### EXPORTAMOS A .CSV EL DATAFRAME ###
os.makedirs('Resultados', exist_ok=True)
dfResultadoT.to_csv('Resultados/TConsulta.csv')

C:\Users\Matias\AppData\Local\Temp\ipykernel_11948\2943533450.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfResultadoT.loc[dfResultadoT['review'] == fila, 'neutro'] = 1          # [...] De lo contrario lo marcamos como neutro.
C:\Users\Matias\AppData\Local\Temp\ipykernel_11948\2943533450.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfResultadoT.loc[dfResultadoT['review'] == fila, 'positi'] = 1          # Si la polaridad es mayor a cero, es positiva.
C:\Users\Matias\AppData\Local\Temp\ipyker

In [30]:
dfResultadoT

,recommend,nueva_fecha,item_id,neutro,positi,negati,item_name
0,False,2010,440,0,1,0,NaN
1,False,2011,105400,1,0,1,Fable III
2,False,2011,107900,1,0,0,War Inc. Battlezone
3,False,2011,18700,0,0,2,And Yet It Moves
4,False,2011,240,0,1,0,Counter-Strike: Source
...,...,...,...,...,...,...,...
6148,True,2015,9900,1,2,1,Star Trek Online
6149,True,2015,9930,0,2,1,Test Drive Unlimited 2
6150,True,2015,99810,1,0,0,Bulletstorm
6151,True,2015,99900,8,7,2,Spiral Knights


In [31]:
#######################
### QUINTA CONSULTA ###
#######################

dfResultadoQ = pd.merge(dfGames, dfResultadoT, how = 'inner', left_on='id', right_on = 'item_id')            # Creamos un nuevo dataframe, agregando a la consulta utilizada para el punto anterior la fecha de lanzamiento de los juegos.
dfResultadoQ = dfResultadoQ.drop(['genres', 'item_id', 'app_name', 'tags', 'id', 'recommend', 'nueva_fecha'], axis = 1)           # Eliminamos columnas que no son necesarias.
dfResultadoQ = dfResultadoQ.groupby('release_date').sum().reset_index()         # Agrupamos por año y sumamos los analisis.
dfResultadoQ['release_date'] = dfResultadoQ['release_date'].astype('int')           # Convertimos los años en formato númerico.

### EXPORTAMOS A .CSV EL DATAFRAME ###
os.makedirs('Resultados', exist_ok = True)
dfResultadoQ.to_csv('Resultados/QConsulta.csv')

In [32]:
dfResultadoQ.head()

,release_date,neutro,positi,negati,item_name
0,1989,0,2,0,Sword of the SamuraiSword of the Samurai
1,1990,1,4,0,LoomCommander Keen Complete PackCommander Keen...
2,1991,0,1,0,Crystal Caves
3,1992,4,2,0,Putt-Putt Joins the ParadePutt-Putt Joins the ...
4,1993,1,2,3,X-COM: UFO DefenseX-COM: UFO DefensePutt-Putt ...
